In [20]:
import numpy as np
import pandas as pd
import math

import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

from keras.layers import BatchNormalization

from keras.optimizers import Adam
from keras.optimizers import SGD

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [3]:
train = pd.read_csv('./data/train.csv', index_col='id')

In [58]:
len(train['type'].unique())

19

In [4]:
# keras 머신러닝 분류 예제
# https://imasoftwareengineer.tistory.com/26?category=771295
# https://talkingaboutme.tistory.com/entry/DL-%ED%95%B4%EB%B3%B4%EB%A9%B4%EC%84%9C-%EB%B0%B0%EC%9A%B0%EB%8A%94-%EB%94%A5%EB%9F%AC%EB%8B%9D-ANN-%EA%B5%AC%ED%98%84-2
# optimizer
# https://sacko.tistory.com/42

In [5]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [6]:
scaler = preprocessing.StandardScaler()

In [7]:
# train columns 가져오는 작업
columns = list(train.columns)
columns = list(filter(lambda x:x != 'type', columns))
len(columns)

21

In [8]:
# 스케일 작업
# 평균과 표준편차를 0 ,1 로 맞춰주는 작업을 한다.
trainDrop = train[columns]
scaled = scaler.fit_transform(trainDrop)

In [9]:
trainScaled = pd.DataFrame(scaled, columns=columns, index=trainDrop.index)
trainScaled['type'] = train['type']
trainScaled.head()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type
id,,,,,,,,,,,,,,,,,,,,,
0,1.065975,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,0.028019,0.026184,...,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607,QSO
1,1.895960,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,0.020351,0.022507,...,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857,QSO
2,0.293690,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,-0.023707,-0.008885,...,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743,QSO
3,2.233280,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,0.012031,0.015448,...,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680,QSO
4,1.114798,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,0.028965,0.013715,...,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874,STAR_RED_DWARF


In [10]:
unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}
labels = train['type']

In [11]:
unique_labels

array(['QSO', 'STAR_RED_DWARF', 'SERENDIPITY_BLUE', 'STAR_BHB',
       'STAR_CATY_VAR', 'SERENDIPITY_DISTANT', 'GALAXY',
       'SPECTROPHOTO_STD', 'REDDEN_STD', 'ROSAT_D', 'STAR_WHITE_DWARF',
       'SERENDIPITY_RED', 'STAR_CARBON', 'SERENDIPITY_FIRST',
       'STAR_BROWN_DWARF', 'STAR_SUB_DWARF', 'SKY', 'SERENDIPITY_MANUAL',
       'STAR_PN'], dtype=object)

In [45]:
trainScaled['type'] = train['type']
trainScaled['type'] = trainScaled['type'].replace(label_dict)
trainScaled.head()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type
id,,,,,,,,,,,,,,,,,,,,,
0,1.065975,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,0.028019,0.026184,...,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607,0
1,1.895960,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,0.020351,0.022507,...,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857,0
2,0.293690,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,-0.023707,-0.008885,...,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743,0
3,2.233280,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,0.012031,0.015448,...,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680,0
4,1.114798,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,0.028965,0.013715,...,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874,1


In [37]:
# trainScaled['type'] = trainScaled['type'].replace(i2lb)
# trainScaled.head()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type
id,,,,,,,,,,,,,,,,,,,,,
0,1.065975,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,0.028019,0.026184,...,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607,QSO
1,1.895960,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,0.020351,0.022507,...,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857,QSO
2,0.293690,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,-0.023707,-0.008885,...,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743,QSO
3,2.233280,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,0.012031,0.015448,...,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680,QSO
4,1.114798,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,0.028965,0.013715,...,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874,STAR_RED_DWARF


In [46]:
from keras import models
from keras import layers

In [47]:
features_train = columns
target_train = ['type']

In [48]:
X = trainScaled[features_train]
y = trainScaled[target_train]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)

In [50]:
# X_train, X_test, y_train, y_test
X_train.shape

(139993, 21)

In [92]:
network=models.Sequential()
network.add(layers.Dense(units=16, kernel_initializer='he_normal', activation='relu', input_shape=(21,)))
network.add(layers.Dense(units=1, kernel_initializer='he_normal', activation='sigmoid'))
# network.add(layers.Dense(units=16, kernel_initializer='he_normal', activation='relu', input_shape=(21,)))
# network.add(layers.Dense(units=1, kernel_initializer='he_normal', activation='softmax'))


network.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [93]:
history = network.fit(
    X_train,
    y_train,
    epochs=100, # 에폭 횟수
    verbose=1, # 에폭 과정을 출력합니다.
    batch_size=100, # 배치의 샘플 개수
)

Epoch 1/100
139993/139993 [==============================] - 2s 17us/step - loss: -68.0807 - accuracy: 0.0722
Epoch 2/100
139993/139993 [==============================] - 2s 17us/step - loss: -488.6348 - accuracy: 0.0698
Epoch 3/100
139993/139993 [==============================] - 2s 17us/step - loss: -1232.8551 - accuracy: 0.0987
Epoch 4/100
139993/139993 [==============================] - 2s 17us/step - loss: -2229.3720 - accuracy: 0.1128
Epoch 5/100
139993/139993 [==============================] - 2s 17us/step - loss: -3455.6934 - accuracy: 0.1157
Epoch 6/100
139993/139993 [==============================] - 2s 17us/step - loss: -4898.5874 - accuracy: 0.1177
Epoch 7/100
139993/139993 [==============================] - 2s 17us/step - loss: -6556.2682 - accuracy: 0.1192
Epoch 8/100
139993/139993 [==============================] - 2s 16us/step - loss: -8427.7490 - accuracy: 0.1205
Epoch 9/100
139993/139993 [==============================] - 2s 16us/step - loss: -10509.6839 - accuracy: 0

KeyboardInterrupt: 